In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import LSTM, Dense, Bidirectional, BatchNormalization, Dropout
from keras import optimizers
from sklearn.model_selection import train_test_split
from keras.models import load_model

In [4]:
# load model
model = load_model('weights-improvement-29-0.98.h5',compile=False)
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
# summarize model.
model.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 250)               379000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               25100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 404       
Total params: 414,604
Trainable params: 414,604
Non-trainable params: 0
_________________________________________________________________


In [0]:
import pickle
with open("mapping.pickle", "rb") as f:
    mapping = pickle.load(f)

In [0]:
testing_filename="equations1.csv"

In [0]:
def get_test_data(filename):
  testing = pd.read_csv(filename)
  test_eqs=testing.math[:]
  test_sequences = list()
  for eq in test_eqs:
    # integer encode line
    encoded_seq = [mapping[char] for char in eq]
    # store
    test_sequences.append(encoded_seq)
  test_padded = pad_sequences(test_sequences, padding='post',value=mapping["?"],maxlen=50)
  test_sequences = [to_categorical(x, num_classes=128) for x in test_padded]
  X_predict = np.array(test_sequences)
  return X_predict,test_eqs

In [0]:
X_predict,test_eqs=get_test_data(testing_filename)

In [0]:
y_pred=model.predict_classes(X_predict)

In [0]:
class_names=["Unknown","Simple","Linear Equation for degree one","Fraction"]

In [0]:
y_pred_class=[class_names[3-y] for y in y_pred]

In [0]:
df = pd.DataFrame(list(zip(list(test_eqs), y_pred_class)), 
               columns =['Math', 'Class']) 

In [0]:
export_csv = df.to_csv (r'prediction.csv', index = None, header=True)